In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential,layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import imdb
import numpy as np

In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
from gensim import downloader

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
wv=downloader.load('word2vec-google-news-300')

[=============-------------------------------------] 27.1% 450.2/1662.8MB downloaded

KeyboardInterrupt: 

In [ ]:
(x_train,y_train),(x_test,y_test)=imdb.load_data()

In [ ]:
x_train[0]
y_train[9]

0

In [ ]:
word2index=imdb.get_word_index()

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
index2word={index:word for word,index in word2index.items()}
index2word

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [ ]:
def decode_review(encoded_review):
  return ' '.join([index2word.get(i,"?") for i in encoded_review])

In [ ]:
x_train_text=[decode_review(encoded_review) for encoded_review in x_train]
x_test_text=[decode_review(encoded_review) for encoded_review in x_test]

In [ ]:
x_train_text[0]

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room titillate it so heart shows to years of every never going villaronga help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of gilmore's br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but wh

In [ ]:
def sentence_embeddings(sentence):
  words=sentence.split()
  vectors=[wv[word] for word in words if word in wv.key_to_index]
  return np.mean(vectors, axis=0) if vectors else np.zeros(300)


In [ ]:
train_embeddings=np.array([sentence_embeddings(sentence) for sentence in x_train_text])
test_embeddings=np.array([sentence_embeddings(sentence) for sentence in x_test_text])


In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((train_embeddings,y_train)).batch(32)
test_dataset=tf.data.Dataset.from_tensor_slices((test_embeddings,y_test)).batch(32)

In [ ]:
model=Sequential(
    [
        #layers.Input(shape=(300,)),
        Dense(256,activation='relu'),
        Dense(128,activation='relu'),
        Dense(1,activation='sigmoid')

    ]
)

In [ ]:
model.compile(loss=keras.losses.BinaryCrossentropy(3e-4),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset,validation_data=test_dataset,epochs=20,verbose=1)

Epoch 1/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7282 - loss: 0.5431 - val_accuracy: 0.7146 - val_loss: 0.5620
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7285 - loss: 0.5426 - val_accuracy: 0.7128 - val_loss: 0.5622
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7280 - loss: 0.5415 - val_accuracy: 0.7126 - val_loss: 0.5630
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7298 - loss: 0.5387 - val_accuracy: 0.7129 - val_loss: 0.5624
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.7305 - loss: 0.5359 - val_accuracy: 0.7117 - val_loss: 0.5654
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.7349 - loss: 0.5332 - val_accuracy: 0.7108 - val_loss: 0.5636
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7369 - loss: 0.5293 - val_accuracy: 0.7126 - val_loss: 0.5629
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7377 - loss: 0.5276 - val_accuracy: 0.

2. Using TensorFlow Hub Universal Sentence Encoder

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
train_embeddings_1 = np.array(embed(x_train_text))
test_embeddings_1= np.array(embed(x_test_text))

ResourceExhaustedError: Graph execution error:

Detected at node EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2 defined at (most recent call last):
<stack traces unavailable>
OOM when allocating tensor with shape[12008826,320] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_restored_function_body_4560]

In [ ]:
import numpy as np
import tensorflow_hub as hub


def embed_in_batches(texts, batch_size=1000):
  """Embeds texts in batches to avoid OOM errors."""
  embeddings = []
  for i in range(0, len(texts), batch_size):
    batch = texts[i : i + batch_size]  # Process a slice of the data
    batch_embeddings = embed(batch)
    embeddings.extend(batch_embeddings)
  return np.array(embeddings)

# Embed the training and testing data in batches
train_embeddings_1 = embed_in_batches(x_train_text)
test_embeddings_1 = embed_in_batches(x_test_text)

In [ ]:
train_embeddings_1.shape

(25000, 512)

In [ ]:
model1=Sequential(
    [
        layers.Input(shape=(512,)),
        Dense(256,activation='relu'),
        Dense(128,activation='relu'),
        Dense(1,activation='sigmoid')

    ]
)

In [ ]:
model1.compile(loss=keras.losses.BinaryCrossentropy(3e-4),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model1.fit(train_embeddings_1,y_train,validation_data=(test_embeddings_1,y_test),batch_size=32,epochs=20,verbose=1)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/nn.py:780: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - accuracy: 0.6205 - loss: 0.6469 - val_accuracy: 0.6574 - val_loss: 0.6162
Epoch 2/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.6689 - loss: 0.6074 - val_accuracy: 0.6595 - val_loss: 0.6164
Epoch 3/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6742 - loss: 0.6008 - val_accuracy: 0.6651 - val_loss: 0.6103
Epoch 4/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6906 - loss: 0.5799 - val_accuracy: 0.6652 - val_loss: 0.6107
Epoch 5/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7110 - loss: 0.5560 - val_accuracy: 0.6625 - val_loss: 0.6143
Epoch 6/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7370 - loss: 0.5237 - val_accuracy: 0.6496 - val_loss: 0.6509
Epoch 7/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7595 - loss: 0.4868 - val_accuracy: 0.6564 - val_loss: 0.6832
Epoch 8/20
782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7871 - loss: 0.4465 - val_accuracy: 0.6519 - val_

3. Using TensorFlow Hub Embeddings

In [ ]:
embedding =hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1")


In [ ]:
def sentence_embeddings1(sentence):
  words=sentence.split()
  vectors=[embedding([word])[0] for word in words if embedding([word])[0] is not None]
  return np.mean(vectors, axis=0) if vectors else np.zeros(300)


In [ ]:
train_embeddings_2=np.array([sentence_embeddings1(sentence) for sentence in x_train_text])
test_embeddings_2=np.array([sentence_embeddings1(sentence) for sentence in x_test_text])


In [ ]:
model2=Sequential(
    [
        layers.Input(shape=(20,)),
        Dense(256,activation='relu'),
        Dense(128,activation='relu'),
        Dense(1,activation='sigmoid')

    ]
)

In [ ]:
model2.compile(loss=keras.losses.BinaryCrossentropy(3e-4),
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model2.fit(train_embeddings_2,y_train,validation_data=(test_embeddings_2,y_test),batch_size=32,epochs=20,verbose=1)